# Extracting Renewable Energy Project Coordinates

This notebook helps you extract the most relevant solar and wind energy project coordinates for selected Central African countries using the Global Integrated Power dataset.

**Instructions:**
- Ensure you have the required Excel file in the `input` folder.
- The output will be saved in the `output` folder.
"""


## 1. Import Required Libraries

We start by importing the necessary Python libraries for file handling and data processing.


In [19]:
import os
import pandas as pd


## 2. Define Target Countries

Here, we specify the list of countries for which we want to extract renewable energy project data.


In [20]:
countries = ['Angola', 'Burundi', 'Cameroon', 'Central African Republic', 'Chad', 'Republic of the Congo', 'DR Congo', 'Equatorial Guinea', 'Gabon']


## 3. Prepare Input and Output Folders

This cell ensures that the `input` and `output` directories exist.
- Place your Excel data file in the `input` folder.
- Results will be saved in the `output` folder.


In [21]:
folder_input = 'input'
if not os.path.exists(folder_input): os.mkdir(folder_input)
folder_output = 'output'
if not os.path.exists(folder_output): os.mkdir(folder_output)


## 4. Load and Filter the Power Facilities Data

- Loads the Excel file containing power facility data.
- Filters the data to include only the selected countries and renewable technologies (solar and wind).

**Note:**
If the required Excel file is missing, an error will be raised. Please ensure the file exists in the `input` folder.


In [ ]:
path = os.path.join(folder_input, 'Global-Integrated-Power-April-2025.xlsx')
if not os.path.exists(path):
    raise FileNotFoundError(f"File not found: {path}. Please ensure the file exists in the specified directory.")

data_power = pd.read_excel(path, sheet_name='Power facilities', index_col=None, header=[0])

data_power = data_power[data_power['Country/area'].isin(countries)]
data_power = data_power[data_power['Type'].isin(['solar', 'wind'])]


## 5. Extract the Most Relevant Projects

This function selects, for each country and technology (solar/wind), the most relevant project based on:
- Project status (priority: operating > construction > pre-construction)
- Largest capacity (MW) within each status

The resulting table contains the most significant project per country and technology.


In [ ]:
def get_most_relevant_projects(df, countries, tech_types):
    """
    Extract the most relevant energy project for each country and technology.

    Parameters:
    - df: pandas DataFrame with columns ['Country/area', 'Type', 'Status', 'Capacity (MW)',
                                         'Plant / Project name', 'Latitude', 'Longitude', 'City']
    - countries: list of country names to filter
    - tech_types: list of technology types to check (values from 'Type' column)

    Returns:
    - A DataFrame with the most relevant project per country and technology.
    """

    results = []

    status_priority = ['operating', 'construction', 'pre-construction', 'announced']

    for country in countries:
        for tech in tech_types:
            filtered = df[(df['Country/area'] == country) & (df['Type'] == tech)]

            for status in status_priority:
                sub = filtered[filtered['Status'] == status]
                if not sub.empty:
                    top_project = sub.loc[sub['Capacity (MW)'].idxmax()]
                    results.append({
                        'Country': country,
                        'Type': tech,
                        'Plant / Project name': top_project['Plant / Project name'],
                        'Capacity (MW)': top_project['Capacity (MW)'],
                        'Latitude': top_project['Latitude'],
                        'Longitude': top_project['Longitude'],
                        'City': top_project['City']
                    })
                    break  # stop at the first status with a result

    return pd.DataFrame(results)

projects = get_most_relevant_projects(data_power, countries, ['solar', 'wind'])



## 6. Display and Save Results

- The table below shows the most relevant solar and wind projects for each country.
- The results are also saved as a CSV file in the `output` folder.


In [18]:
display(projects)
projects.to_csv(os.path.join(folder_output, 'most_relevant_projects_solar_wind.csv'), index=False)

,Country,Type,Plant / Project name,Capacity (MW),Latitude,Longitude,City
0,Angola,solar,Biópio solar farm,189.0,-12.4667,15.7442,Biópio
1,Angola,wind,Morro do Ouro wind farm,50.0,-15.2198,12.7258,NaN
2,Burundi,solar,Bubanza SB solar project,8.0,-3.1479,29.3625,Bubanza
3,Cameroon,solar,Garoua solar farm,30.0,9.3271,13.3621,Garoua
4,Cameroon,wind,Cameroon wind farm,100.0,4.8918,12.3598,NaN
5,Central African Republic,solar,Enerca Danzi solar farm,25.0,4.5167,18.4667,Danzi
6,Chad,solar,N'Djamena (Qair) solar farm,21.5,12.0905,15.1331,Gassi District
7,Republic of the Congo,solar,District De Kabo solar project,2.0,1.3863,16.3364,NaN
8,DR Congo,solar,Goma Nyiragongo solar project,1.5,-1.6414,29.1938,Goma
9,Gabon,solar,Ayémé Plaine solar farm,60.0,0.2293,9.5315,Ayémé Plaine
